In [ ]:
__author__ = 'Alex Drlica-Wagner <kadrlica@fnal.gov>' # single string; emails in <>
__version__ = '20190816' # yyyymmdd; version datestamp of this notebook
__datasets__ = ['delve']  # datasets used in this notebook; for available datasets, see cell "Available datasets in Data Lab" further below
__keywords__ = [''], # keywords relevant to this notebook, e.g. ['science case','dwarf galaxies'] Use only keywords from the master list: https://github.com/noaodatalab/notebooks-latest/blob/master/internal/keywords.txt

In [ ]:
%matplotlib inline
from getpass import getpass

import multiprocessing
import glob
# This is vestigial
import logging
logger = logging.getLogger()

import numpy as np
import healpy as hp
import pylab as plt
import fitsio
from matplotlib.colors import LogNorm

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc

In [ ]:
token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))

In [ ]:
token

In [ ]:
def select(data):
    """ Select data from a catalog array. The array must have columns:
    MAG_PSF_G, MAG_PSF_R, EXTINCTION_G, EXTINCTION_R, SPREAD_MODEL_R

    Parameters:
    -----------
    data : input array for selection
    
    Returns:
    --------
    sel  : boolean selection
    """
    # De-redened magnitudes
    g_sfd = (data['MAG_PSF_G']-data['EXTINCTION_G'])
    r_sfd = (data['MAG_PSF_R']-data['EXTINCTION_R'])

    # Detection in g,r cut
    sel  = (g_sfd < 30) & (r_sfd < 30)
    # Color cut
    sel &= (g_sfd - r_sfd) < 0.3
    # Star-Galaxy separation
    sel &= (np.abs(data['SPREAD_MODEL_R']) < 0.003)

    return sel
    
def load(args):
    """ Load a catalog from a single healpix file. 
    
    Parameters:
    -----------
    args : tuple of (infile,columns) passed to fitsio.read
    
    Returns:
    --------
    data : loaded array with selection applied
    """
    infile,columns = args
    logger.debug("Loading %s..."%infile)
    data = fitsio.read(infile,columns=columns)
    # operations on data
    return data[select(data)]

def load_infiles(infiles,columns=None,multiproc=False):
    """ Load multiple input files.
    
    Parameters:
    -----------
    infiles   : list of input fits files
    columns   : list of columns to load
    multiproc : number of cores to execute on
    
    Returns:
    --------
    data : return numpy array 
    """
    if isinstance(infiles,str):
        infiles = [infiles]

    logger.debug("Loading %s files..."%len(infiles))

    args = list(zip(infiles,len(infiles)*[columns]))

    if multiproc:
        from multiprocessing import Pool
        processes = multiproc if multiproc > 0 else None
        p = Pool(processes,maxtasksperchild=1)
        out = p.map(load,args)
    else:
        out = [load(arg) for arg in args]

    dtype = out[0].dtype
    for i,d in enumerate(out):
        if d.dtype != dtype: 
            # ADW: Not really safe...
            logger.warn("Casting input data to same type.")
            out[i] = d.astype(dtype)

    logger.debug('Concatenating arrays...')
    return np.concatenate(out)

In [ ]:
filename = 'delve_wide_blue_stars_y1t2.fits.gz'
if os.path.exists(filename):
    data = fitsio.read(filename)
else:
    files = sorted(glob.glob('../DELVE/y1t2/*.fits'))
    columns = ['RA','DEC','MAG_PSF_G','EXTINCTION_G','MAG_PSF_R','EXTINCTION_R','SPREAD_MODEL_R']
    data = load_infiles(files,columns=columns,multiproc=16)
    fitsio.write(filename,data)

In [ ]:
nside = 64
#hpxmap = np.zeros(hp.nside2npix(nside))
hpxmap = hp.UNSEEN*np.ones(hp.nside2npix(nside))
pixel = hp.ang2pix(nside,data['RA'],data['DEC'],lonlat=True)
pix, cts = np.unique(pixel,return_counts=True)
hpxmap[pix] = cts

In [ ]:
vmin,vmax = np.percentile(hpxmap[hpxmap>0],[5,95])
hp.mollview(hpxmap,min=vmin,max=vmax,rot=(180, 0, 0))
ra1,dec1 = (223,-5.5)
ra2,dec2 = (160,-5.5)
hp.projscatter(ra1,dec1,lonlat=True,c='r')
hp.projscatter(ra2,dec2,lonlat=True,c='orange')
hp.graticule()

In [ ]:
hp.cartview(hpxmap,lonra=[-180,-120],latra=[-30,10],max=5000)
hp.projscatter(ra1,dec1,lonlat=True,color='r')
hp.graticule()

In [ ]:
def angsep(lon1,lat1,lon2,lat2):
    """
    Angular separation (deg) between two sky coordinates.
    Borrowed from astropy (www.astropy.org)
    Notes
    -----
    The angular separation is calculated using the Vincenty formula [1],
    which is slighly more complex and computationally expensive than
    some alternatives, but is stable at at all distances, including the
    poles and antipodes.
    [1] http://en.wikipedia.org/wiki/Great-circle_distance
    """
    lon1,lat1 = np.radians([lon1,lat1])
    lon2,lat2 = np.radians([lon2,lat2])
    
    sdlon = np.sin(lon2 - lon1)
    cdlon = np.cos(lon2 - lon1)
    slat1 = np.sin(lat1)
    slat2 = np.sin(lat2)
    clat1 = np.cos(lat1)
    clat2 = np.cos(lat2)

    num1 = clat2 * sdlon
    num2 = clat1 * slat2 - slat1 * clat2 * cdlon
    denominator = slat1 * slat2 + clat1 * clat2 * cdlon

    return np.degrees(np.arctan2(np.hypot(num1,num2), denominator))

In [ ]:
sep = angsep(ra1,dec1,data['RA'],data['DEC'])
d1 = data[sep < 10]

In [ ]:
sep = angsep(ra2,dec2,data['RA'],data['DEC'])
d2 = data[sep < 10]

In [ ]:
g_sfd = (d1['MAG_PSF_G'] - d1['EXTINCTION_G']) 
r_sfd = (d1['MAG_PSF_R'] - d1['EXTINCTION_R'])
bins = [np.linspace(-0.5,0.3),np.linspace(16,24)]
n1,b,e,p = plt.hist2d(g_sfd - r_sfd,g_sfd,bins=bins,norm=LogNorm(),vmax=2e3)
plt.colorbar()
plt.annotate("%.1f, %.1f"%(ra1,dec1),(0.1,0.9),xycoords='axes fraction',
            bbox=dict(facecolor='w'))
plt.xlabel("g - r"); plt.ylabel("g")
plt.gca().invert_yaxis()

In [ ]:
g_sfd = (d2['MAG_PSF_G'] - d2['EXTINCTION_G']) 
r_sfd = (d2['MAG_PSF_R'] - d2['EXTINCTION_R'])
bins = [np.linspace(-0.5,0.3),np.linspace(16,24)]
n2,b,e,p = plt.hist2d(g_sfd - r_sfd,g_sfd,bins=bins,norm=LogNorm(),vmax=2e3)
plt.colorbar()
plt.annotate("%.1f, %.1f"%(ra2,dec2),(0.1,0.9),xycoords='axes fraction',
            bbox=dict(facecolor='w'))
plt.xlabel("g - r"); plt.ylabel("g")
plt.gca().invert_yaxis()

In [ ]:
plt.pcolormesh(bins[0],bins[1],(n1-n2).T,norm=LogNorm())
plt.colorbar()
plt.xlabel("g - r"); plt.ylabel("g")
plt.gca().invert_yaxis()

In [ ]:
#filename = 'delve_wide_blue_stars_y1t2.fits.gz'
#fitsio.write(filename,data)

#hpxmap_filename = 'delve_wide_blue_stars_y1t2_ring_n%i.fits.gz'%nside
#hp.write_map(hpxmap_filename,hpxmap)

# To copy the notebook somewhere that others can access
```
sc.put('DELVE_BlueStars_Y1T2.ipynb','vos://public/')
```

In [ ]:
#sc.put('delve_wide_blue_stars_y1t2_ring_n%i.fits.gz'%nside,'vos://public/')

In [ ]:
#sc.put(fr='./delve_wide_blue_stars_y1t2_ring_n64.fits.gz', to='vos://public')

In [ ]:
#sc.put(fr='DELVE_BlueStars_Y1T2.ipynb', to='vos://public')

In [ ]:
#sc.ls('vos://public')

In [ ]:
#sc.ls('kadrlica://public')

In [ ]:
#sc.get(fr='kadrlica://public/delve_wide_blue_stars_y1t2.fits.gz', to='./tmp.fits.gz')